<a href="https://colab.research.google.com/github/PUGHAZH2005/GEE/blob/main/Flood_risk_zones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "geemap[ai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 320.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 

In [2]:
!pip install earthengine-api

In [3]:
# Install Folium for visualization
!pip install folium
import folium


In [4]:
import geemap

In [5]:
import ee
ee.Authenticate()
ee.Initialize (project='ee-rvsgkl20')

In [17]:
# prompt: CRAETE SURFACE RUNOFF FOR CHENNAI USING A LAND USE AND LANDCOVER DATA FREE AVAILABLLE,CLIP AN ROI BASED ON ADMINISTRATIVE BOUNDARY OF DISTRICT CHENNAI AND CLIP TO ROI

# Import the required libraries
import geemap
import ee

# Define the region of interest (ROI) - Chennai district boundary
# Replace with actual coordinates or a FeatureCollection if available
chennai_roi = ee.Geometry.Polygon(
    [[[80.06, 13.23],
      [80.35, 13.23],
      [80.35, 12.90],
      [80.06, 12.90]]])

# Load the land use/land cover data
# Replace with the actual dataset ID from the GEE catalog
# 'ESA/WorldCover/v200' is an ImageCollection, select the 'Map' band to get an Image
# The band name is case-sensitive, so it should be 'Map', not 'map'
lulc = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map').clip(chennai_roi)

# Define a function to calculate surface runoff based on LULC class
def calculate_runoff(image):
  # Example: Assign runoff coefficients based on LULC classes
  # This needs to be adapted based on the specific LULC dataset and local conditions
  runoff_coeffs = {
      10: 0.1,  # Tree cover
      20: 0.2,  # Shrubland
      30: 0.3,  # Grassland
      40: 0.5,  # Cropland
      50: 0.7,  # Built-up
      60: 0.8,  # Bare / sparse vegetation
      70: 0.9,  # Snow and ice
      80: 0.0,  # Permanent water bodies
      90: 0.2,  # Herbaceous wetland
      95: 0.2,  # Mangroves
      100: 0.4  # Moss and lichen
      # Add other LULC classes and runoff coefficients as needed
  }

  runoff_image = image.remap(list(runoff_coeffs.keys()),
                             list(runoff_coeffs.values()), 0)
  return runoff_image

# Apply the runoff calculation function to the LULC data
runoff = calculate_runoff(lulc)

# Visualization parameters
vis_params = {
  'min': 0,
  'max': 0.9,
  'palette': ['blue', 'green', 'yellow', 'red']
}

# Add the runoff layer to the map
map = geemap.Map()
map.centerObject(chennai_roi, 10)
map.addLayer(runoff, vis_params, 'Surface Runoff')
map.addLayerControl()
map

Map(center=[13.065003645802529, 80.20499999999912], controls=(WidgetControl(options=['position', 'transparent_…

In [87]:
# Import the required libraries
import geemap
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Step 1: Input AOI (Load AOI from a FeatureCollection)
# Load the FeatureCollection
table = ee.FeatureCollection('projects/ee-rvsgkl20/assets/INDIA_TALUK')  # Replace with your FeatureCollection path
chennai_roi = table.filter(ee.Filter.eq('District', 'CHENNAI'))  # Change as needed

# Step 2: Check if AOI is valid
if roi.size().getInfo() == 0:
    print('No features found for the specified district. Please check the district name and FeatureCollection.')
else:
    # Set the map center and add AOI layer
    Map = geemap.Map(center=[13.11, 80.22], zoom=10)  # Centering on Coimbatore

# Load the land use/land cover data
lulc = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map').clip(chennai_roi)

# Define a function to calculate surface runoff based on LULC class
def calculate_runoff(image):
    runoff_coeffs = {
        10: 0.1,  # Tree cover
        20: 0.2,  # Shrubland
        30: 0.3,  # Grassland
        40: 0.5,  # Cropland
        50: 0.7,  # Built-up
        60: 0.8,  # Bare / sparse vegetation
        70: 0.9,  # Snow and ice
        80: 0.0,  # Permanent water bodies
        90: 0.2,  # Herbaceous wetland
        95: 0.2,  # Mangroves
        100: 0.4  # Moss and lichen
    }
    runoff_image = image.remap(list(runoff_coeffs.keys()), list(runoff_coeffs.values()), 0)
    return runoff_image

# Apply the runoff calculation function to the LULC data
runoff = calculate_runoff(lulc)

# Load DEM data (using SRTM as an example)
dem = ee.Image('USGS/SRTMGL1_003').clip(chennai_roi)

# Define a threshold for flood-prone areas based on elevation (example)
flood_threshold = 10  # meters

# Create a flood zone mask based on elevation
flood_zone = dem.lt(flood_threshold)  # Areas below the threshold are considered flood-prone

# Combine runoff and flood zone to develop a flood risk zone
flood_risk = runoff.multiply(flood_zone)

# Classify zones based on runoff values
high_risk_zones = flood_risk.updateMask(runoff.gt(0.7))
low_risk_zones = flood_risk.updateMask(runoff.lte(0.4))

# Create distinct high-risk zones using connected components
high_risk_zones_connected = high_risk_zones.lt(0.8)  # Create a binary image for high risk
high_risk_zones_components = high_risk_zones_connected.connectedComponents(
    connectedness=ee.Kernel.square(1),  # Use a Kernel object for 4-connectedness
    maxSize=128
)

# Count distinct high-risk zones
num_high_risk_zones = high_risk_zones_components.select('labels').reduceRegion(
    reducer=ee.Reducer.countDistinct(),  # Use countDistinct to count unique labels
    geometry=chennai_roi,
    scale=30
).get('labels')



# Create distinct low-risk zones using connected components
low_risk_zones_connected = low_risk_zones.lt(0.8)  # Create a binary image for low risk
low_risk_zones_components = low_risk_zones_connected.connectedComponents(
    connectedness=ee.Kernel.square(1),  # Use a Kernel object for 4-connectedness
    maxSize=128
)

# Count distinct low-risk zones
num_low_risk_zones = low_risk_zones_components.select('labels').reduceRegion(
    reducer=ee.Reducer.countDistinct(),  # Use countDistinct to count unique labels
    geometry=chennai_roi,
    scale=30
).get('labels')


# Print the counts of zones
print('Number of High Risk Zones:', num_high_risk_zones.getInfo())
print('Number of Low Risk Zones:', num_low_risk_zones.getInfo())
# Print flood height statistics for each zone
# Zone 1: High Risk (runoff > 0.7)
zone1_height = flood_height.updateMask(runoff.gt(0.6)).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=chennai_roi,
    scale=30,
    maxPixels=1e13
).get('remapped')

# Zone 2: Moderate Risk (0.4 < runoff <= 0.7)
zone2_height = flood_height.updateMask(runoff.gt(0.4).And(runoff.lte(0.6))).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=chennai_roi,
    scale=30,
    maxPixels=1e13
).get('remapped')

# Zone 3: Low Risk (runoff <= 0.4)
zone3_height = flood_height.updateMask(runoff.lte(0.4)).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=chennai_roi,
    scale=30,
    maxPixels=1e13
).get('remapped')

# Print the results
print('Flood Height in High Risk Zone:', zone1_height.getInfo())
print('Flood Height in Moderate Risk Zone:', zone2_height.getInfo())
print('Flood Height in Low Risk Zone:', zone3_height.getInfo())

# Visualization parameters
vis_params_runoff = {
    'min': 0,
    'max': 0.8,
    'palette': ['blue', 'green', 'yellow', 'red']
}

vis_params_flood_risk = {
    'min': 0,
    'max': flood_threshold,
    'palette': ['blue', 'cyan', 'yellow', 'red']
}

# Create a map
Map = geemap.Map(center=[13.11, 80.22], zoom=10)

# Add layers to the map
Map.addLayer(chennai_roi, {'color':'black'}, 'Area of Interest (AOI)')
Map.addLayer(runoff, vis_params_runoff, 'Runoff Coefficients')
Map.addLayer(dem, {'min': -42, 'max': 154, 'palette': ['blue', 'lightgreen', 'yellow', 'red']}, 'DEM')
Map.addLayer(flood_zone, {'palette': ['green','yellow','red']}, 'Flood Zone (Below Threshold)')
Map.addLayer(high_risk_zones_connected, {'palette': ['red']}, 'High Risk Zones')
Map.addLayer(low_risk_zones_connected, {'palette': ['green']}, 'Low Risk Zones')

# Display the map
Map


Number of High Risk Zones: 1689
Number of Low Risk Zones: 6201
Flood Height in High Risk Zone: 7.02511024316767
Flood Height in Moderate Risk Zone: 5
Flood Height in Low Risk Zone: 1.1724588052633702


Map(center=[13.11, 80.22], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…